# Excercises 
# 1. Tune the network
Run the experiment below, explore the different parameters (see suggestions below) and study the result with tensorboard. 
Make a single page (1 a4) report of your findings. Use your visualisation skills to communicate your most important findings.

In [10]:
import sys
print(sys.executable)

c:\Users\Alex\AppData\Local\Programs\Python\Python311\python.exe


In [2]:
from mads_datasets import DatasetFactoryProvider, DatasetType

from mltrainer.preprocessors import BasePreprocessor
from mltrainer import imagemodels, Trainer, TrainerSettings, ReportTypes, metrics

import torch.optim as optim
from torch import nn
from tomlserializer import TOMLSerializer

We will be using `tomlserializer` to easily keep track of our experiments, and to easily save the different things we did during our experiments.
It can export things like settings and models to a simple `toml` file, which can be easily shared, checked and modified.

First, we need the data. 

In [3]:
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
preprocessor = BasePreprocessor()
streamers = fashionfactory.create_datastreamer(batchsize=64, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

2026-02-22 13:10:23.343 | INFO     | mads_datasets.base:download_data:94 - Start download...
  0%|          | 0.00/55.4M [00:00<?, ?iB/s]2026-02-22 13:10:26.330 | INFO     | mads_datasets.datatools:get_file:105 - Downloading C:\Users\Alex\.cache\mads_datasets\fashionmnist\fashionmnist.pt
100%|██████████| 55.4M/55.4M [00:02<00:00, 23.9MiB/s]
2026-02-22 13:10:28.725 | INFO     | mads_datasets.base:download_data:112 - Digest of C:\Users\Alex\.cache\mads_datasets\fashionmnist\fashionmnist.pt matches expected digest


We need a way to determine how well our model is performing. We will use accuracy as a metric.

In [5]:
accuracy = metrics.Accuracy()

You can set up a single experiment.

- We will show the model batches of 64 images, 
- and for every epoch we will show the model 100 batches (trainsteps=100).
- then, we will test how well the model is doing on unseen data (teststeps=100).
- we will report our results during training to tensorboard, and report all configuration to a toml file.
- we will log the results into a directory called "modellogs", but you could change this to whatever you want.

In [6]:
import torch
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=100,
    valid_steps=100,
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
)


2026-02-22 13:10:56.973 | INFO     | mltrainer.settings:check_path:60 - Created logdir c:\Users\Alex\Documents\GitHub\portfolio-example\1-hypertuning-gridsearch\modellogs


We will use a very basic model: a model with three linear layers.

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            nn.Linear(units2, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork(
    num_classes=10, units1=256, units2=256)

I developped the `tomlserializer` package, it is a useful tool to save configs, models and settings as a tomlfile; that way it is easy to track what you changed during your experiments.

This package will 1. check if there is a `__dict__` attribute available, and if so, it will use that to extract the parameters that do not start with an underscore, like this:

In [8]:
{k: v for k, v in model.__dict__.items() if not k.startswith("_")}

{'training': True, 'num_classes': 10, 'units1': 256, 'units2': 256}

This means that if you want to add more parameters to the `.toml` file, eg `units3`, you can add them to the class like this:

```python
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int, units3: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.units3 = units3  # <-- add this line
```

And then it will be added to the `.toml` file. Check the result for yourself by using the `.save()` method of the `TomlSerializer` class like this:

In [9]:
tomlserializer = TOMLSerializer()
tomlserializer.save(settings, "settings.toml")
tomlserializer.save(model, "model.toml")

Check the `settings.toml` and `model.toml` files to see what is in there.

You can use the `Trainer` class from my `mltrainer` module to train your model. It has the TOMLserializer integrated, so it will automatically save the settings and model to a toml file if you have added `TOML` as a reporttype in the settings.

In [10]:
trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau
)
trainer.loop()

2026-02-22 13:12:06.261 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\20260222-131206
2026-02-22 13:12:07.784 | INFO     | mltrainer.trainer:__init__:66 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 100/100 [00:00<00:00, 236.10it/s]
2026-02-22 13:12:08.624 | INFO     | mltrainer.trainer:report:215 - Epoch 0 train 0.9283 test 0.6315 metric ['0.7725']
100%|██████████| 100/100 [00:00<00:00, 234.71it/s]
2026-02-22 13:12:09.314 | INFO     | mltrainer.trainer:report:215 - Epoch 1 train 0.5804 test 0.5416 metric ['0.8045']
100%|██████████| 100/100 [00:00<00:00, 217.59it/s]
2026-02-22 13:12:10.073 | INFO     | mltrainer.trainer:report:215 - Epoch 2 train 0.5294 test 0.5683 metric ['0.7719']
2026-02-22 13:12:10.073 | INFO     | mltrainer.trainer:__call__:258 - best loss: 0.5416, current loss 0.5683.Counter 1/10.
100%|██████████| 3/3 [00:02<00:00,  1.41it/s]


Now, check in the modellogs directory the results of your experiment.

We can now loop this with a naive approach, called a grid-search (why do you think i call it naive?).

In [11]:
units = [256, 128, 64]
for unit1 in units:
    for unit2 in units:
        print(f"Units: {unit1}, {unit2}")

Units: 256, 256
Units: 256, 128
Units: 256, 64
Units: 128, 256
Units: 128, 128
Units: 128, 64
Units: 64, 256
Units: 64, 128
Units: 64, 64


Of course, this might not be the best way to search for a model; some configurations will be better than others (can you predict up front what will be the best configuration?).

So, feel free to improve upon the gridsearch by adding your own logic.

In [ ]:
import torch
from itertools import product

# --- Pas hier je experiment waardes aan ---
units = [256, 128, 64]           # units1 en units2
epochs_list = [5, 10]            # aantal epochs
batchsizes = [32, 64]            # batchsize
extra_layers = [True, False]     # extra layer toevoegen?
learning_rates = [1e-3, 1e-4]    # learning rate
optimizers = ['SGD', 'Adam']     # optimizer keuze
# -------------------------------------

loss_fn = torch.nn.CrossEntropyLoss()

for unit1, unit2, epochs, batchsize, extra_layer, lr, opt_name in product(
    units, units, epochs_list, batchsizes, extra_layers, learning_rates, optimizers
):
    # Maak model, pas eventueel extra_layer parameter aan
    model = NeuralNetwork(num_classes=10, units1=unit1, units2=unit2, extra_layer=extra_layer)

    # Maak optimizer
    if opt_name == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    else:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

    # Settings per run
    settings = TrainerSettings(
        epochs=epochs,
        metrics=[accuracy],
        logdir=f"modellogs/u1_{unit1}_u2_{unit2}_ep_{epochs}_bs_{batchsize}_lr_{lr}_{opt_name}",
        train_steps=len(train),
        valid_steps=len(valid),
        reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
    )

    # Trainer
    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optimizer,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=scheduler
    )

    trainer.loop()


2026-02-22 13:21:31.489 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\20260222-132131
2026-02-22 13:21:31.490 | INFO     | mltrainer.trainer:__init__:66 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 937/937 [00:03<00:00, 237.01it/s]
2026-02-22 13:21:35.867 | INFO     | mltrainer.trainer:report:215 - Epoch 0 train 0.5134 test 0.4428 metric ['0.8322']
100%|██████████| 937/937 [00:03<00:00, 236.02it/s]
2026-02-22 13:21:40.235 | INFO     | mltrainer.trainer:report:215 - Epoch 1 train 0.3640 test 0.3864 metric ['0.8597']
100%|██████████| 937/937 [00:04<00:00, 233.57it/s]
2026-02-22 13:21:44.644 | INFO     | mltrainer.trainer:report:215 - Epoch 2 train 0.3290 test 0.3791 metric ['0.8617']
100%|██████████| 937/937 [00:04<00:00, 211.29it/s]
2026-02-22 13:21:49.548 | INFO     | mltrainer.trainer:report:215 - Epoch 3 train 0.3048 test 0.3588 metric ['0.8726']
100%|██████████| 937/937 [00:04<00:00, 205.38it/s]
2026-0

Because we have set the ReportType to TOML, you will find in every log dir a model.toml and settings.toml file.

Run the experiment, and study the result with tensorboard. 

Locally, it is easy to do that with VS code itself. On the server, you have to take these steps:

- in the terminal, `cd` to the location of the repository
- activate the python environment for the shell. Note how the correct environment is being activated.
- run `tensorboard --logdir=modellogs` in the terminal
- tensorboard will launch at `localhost:6006` and vscode will notify you that the port is forwarded
- you can either press the `launch` button in VScode or open your local browser at `localhost:6006`